# Working with Text Data and Naive Bayes in scikit-learn

## Agenda

**Working with text data**

- Representing text as data
- Reading SMS data
- Vectorizing SMS data
- Examining the tokens and their counts
- Bonus: Calculating the "spamminess" of each token

**Naive Bayes classification**

- Building a Naive Bayes model
- Comparing Naive Bayes with logistic regression

## Part 1: Representing text as data

From the [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> Text Analysis is a major application field for machine learning algorithms. However the raw data, a sequence of symbols cannot be fed directly to the algorithms themselves as most of them expect **numerical feature vectors with a fixed size** rather than the **raw text documents with variable length**.

We will use [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) to "convert text into a matrix of token counts":

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# start with a simple example
simple_train = ['call you tonight', 'Call me a cab', 'please call me... PLEASE!']

In [ ]:
simple_train

In [ ]:
# learn the 'vocabulary' of the training data
vect = CountVectorizer()
vect.fit(simple_train)

vect.get_feature_names()

- binary = True --> flag for absence or presence of a word in a TDM
- ngram_

In [ ]:
vect

# 'ngram_range' : The size of n_gram to make
# binary : tells you only if the word is present, doesnot gives the frequency
# stop_words : to specify incase stop words are not required

In [ ]:
CountVectorizer?

In [ ]:
# transform training data into a 'document-term matrix'
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm

In [ ]:
# print the sparse matrix


# This is a sparse matrix and returns [(row_i,col_i) freq if freq > 0]
print(simple_train_dtm)

In [ ]:
# convert sparse matrix to a dense matrix
print(simple_train_dtm.toarray().__class__)
print(simple_train_dtm.toarray(),'\n')

print(simple_train_dtm.todense().__class__)
print(simple_train_dtm.todense())

In [ ]:
# examine the vocabulary and document-term matrix together
import pandas as pd
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

From the [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> In this scheme, features and samples are defined as follows:

> - Each individual **token** occurrence frequency (normalized or not) is treated as a **feature**.
> - The vector of all the token frequencies for a given document is considered a multivariate **sample**.

> A **corpus of documents** can thus be represented by a matrix with **one row per document** and **one column per token** (e.g. word) occurring in the corpus.

> We call **vectorization** the general process of turning a collection of text documents into numerical feature vectors. This specific strategy (tokenization, counting and normalization) is called the **Bag of Words** or "Bag of n-grams" representation. Documents are described by word occurrences while completely ignoring the relative position information of the words in the document.

In [ ]:
# transform testing data into a document-term matrix (using existing vocabulary)
simple_test = ["please don't call me"]
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

In [ ]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names())

**Summary:**

- `vect.fit(train)` learns the vocabulary of the training data
- `vect.transform(train)` uses the fitted vocabulary to build a document-term matrix from the training data
- `vect.transform(test)` uses the fitted vocabulary to build a document-term matrix from the testing data (and ignores tokens it hasn't seen before)

## Part 2: Reading SMS data

In [ ]:
sms = pd.read_csv('../input/sms.csv')

In [ ]:
sms.head()

In [ ]:
sms.label.value_counts()

In [ ]:
# convert label to a numeric variable
sms['label'] = sms.label.map({'ham':0, 'spam':1})

In [ ]:
# define X and y
X = sms.message
y = sms.label

In [ ]:
# split into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)

## Part 3: Vectorizing SMS data

In [ ]:
# instantiate the vectorizer
vect = CountVectorizer()

In [ ]:
# learn training data vocabulary, then create document-term matrix
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

In [ ]:
# alternative: combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm

In [ ]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

## Part 4: Examining the tokens and their counts

In [ ]:
# store token names
X_train_tokens = vect.get_feature_names()

string = list(filter(lambda x:x.isalpha(),X_train_tokens)) # filtering only string tokens

numeric = list(filter(lambda x:x.isnumeric(),X_train_tokens)) # filtering only numeric tokens

string[:10]

In [ ]:
# view X_train_dtm as a dense matrix
X_train_dtm.toarray()

In [ ]:
# count how many times EACH token appears across ALL messages in X_train_dtm

# sum for each column(vector)
import numpy as np
X_train_counts = np.sum(X_train_dtm.toarray(), axis=0)
X_train_counts

In [ ]:
X_train_counts.shape  # Shape here is 7456 coz we are only learning only from x_train
                      # whereas, this shape would change to 8713 vectors incase if we are learning(fitting) all values from the whole sms dataset

In [ ]:
# create a DataFrame of tokens with their counts - X_train 

pd.DataFrame({'token': X_train_tokens, 'count':X_train_counts}).head()

### Bonus: Calculating the "spamminess" of each token

- This is for the whole dataset

In [ ]:
# create separate DataFrames for ham and spam

sms_ham = sms[sms.label==0]
sms_spam = sms[sms.label==1]

- The model would learn the vectors(when we call fit) on the whole sms dataset
- After that when we call tranform(for ham and spam separately), the model would keep all the vectors but will calc the frequency separately for hama and spam

In [ ]:
# learn the vocabulary of ALL messages and save it
vect.fit(sms.message)
all_tokens = vect.get_feature_names()

In [ ]:
# create document-term matrices for ham and spam
ham_dtm = vect.transform(sms_ham.message)
spam_dtm = vect.transform(sms_spam.message)

In [ ]:
# count how many times EACH token appears across ALL ham messages
ham_counts = np.sum(ham_dtm.toarray(), axis=0)

print(ham_counts.shape)
ham_counts

In [ ]:
# count how many times EACH token appears across ALL spam messages
spam_counts = np.sum(spam_dtm.toarray(), axis=0)

print(spam_counts.shape)
spam_counts

In [ ]:
# create a DataFrame of tokens with their separate ham and spam counts

# This dataframe is made from 'tokens' from vector.fit(sms)
# and then sms.fit_transform(ham and spam) 

token_counts = pd.DataFrame({'token':all_tokens, 'ham':ham_counts, 'spam':spam_counts})
token_counts.head(10)

In [ ]:
# add one to ham and spam counts to avoid dividing by zero (in the step that follows)
token_counts['ham'] = token_counts.ham + 1
token_counts['spam'] = token_counts.spam + 1

In [ ]:
# calculate ratio of spam-to-ham for each token
token_counts['spam_ratio'] = token_counts.spam / token_counts.ham
token_counts.sort_values('spam_ratio').head(10)

## Part 5: Building a Naive Bayes model

We will use [Multinomial Naive Bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html):

> The multinomial Naive Bayes classifier is suitable for classification with **discrete features** (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

In [ ]:
# We have 4179 messages and 7456 vectors

print(X_train_dtm.toarray().shape) # Here the vector space is fromed from training set and transform set is also from training set
print(y_train.shape)

In [ ]:
X_train_dtm.toarray()[:2]

In [ ]:
# train a Naive Bayes model using X_train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)

In [ ]:
# make class predictions for X_test_dtm

print(X_test_dtm.shape)
y_pred_class = nb.predict(X_test_dtm)

In [ ]:
# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

In [ ]:
# confusion matrix
print(metrics.confusion_matrix(y_test, y_pred_class))

In [ ]:
# predict (poorly calibrated) probabilities
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

In [ ]:
# calculate AUC
print(metrics.roc_auc_score(y_test, y_pred_prob))

In [ ]:
# print message text for the false positives
X_test[y_test < y_pred_class]

In [ ]:
# print message text for the false negatives
X_test[y_test > y_pred_class]

In [ ]:
# what do you notice about the false negatives?
X_test[3132]

## Part 6: Comparing Naive Bayes with logistic regression

In [ ]:
# import/instantiate/fit
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train_dtm, y_train)

In [ ]:
# class predictions and predicted probabilities
y_pred_class = logreg.predict(X_test_dtm)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]

In [ ]:
# calculate accuracy and AUC
print(metrics.accuracy_score(y_test, y_pred_class))
print(metrics.roc_auc_score(y_test, y_pred_prob))